
# # 📚 RAG-System mit LangChain, ChromaDB und Gemini 2
# Dieses Notebook implementiert ein einfaches Retrieval-Augmented Generation (RAG) System.
# Es verwendet ChromaDB zur Dokumentenspeicherung, LangChain für Workflow-Management und das Modell `gemini-2.0-flash`.


# 📥 Bibliotheken importieren

In [1]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
#from langchain.embeddings import GooglePalmEmbeddings
from langchain.vectorstores import Chroma
from langchain.memory import ConversationBufferMemory
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import ConversationalRetrievalChain

import os
#from langchain_huggingface import HuggingFaceEmbeddings
#from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
#from langchain.embeddings import HuggingFaceEmbeddings

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
os.environ["USER_AGENT"] = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"

# ## 🌐 Schritt 1: Wikipedia-Seite laden

In [3]:
url = "https://en.wikipedia.org/wiki/2025_in_science"
loader = WebBaseLoader(url)
documents = loader.load()

# ## ✂️ Schritt 2: Text in Chunks aufteilen

In [4]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100
)
chunks = splitter.split_documents(documents)
print(f"Anzahl der Chunks: {len(chunks)}")

Anzahl der Chunks: 89


# ## 🧠 Schritt 3: Vektorisierung und Speicherung in ChromaDB

In [7]:
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

C:\Users\volodymyr\AppData\Local\Temp\ipykernel_26284\4093584008.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
c:\Users\volodymyr\AppData\Local\anaconda3\envs\rag_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
#embedding = GooglePalmEmbeddings()
#embedding =OpenAIEmbeddings()  

persist_directory = "chroma_db"
if os.path.exists(persist_directory) and os.listdir(persist_directory):
    vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)
else:
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embedding,
        persist_directory=persist_directory
    )
    vectorstore.persist()


C:\Users\volodymyr\AppData\Local\Temp\ipykernel_26284\2098072650.py:6: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(persist_directory=persist_directory, embedding_function=embedding)



# ## 💬 Schritt 4: Dialogsystem mit Gedächtnis (Memory)

In [9]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory
)

# ## 🧪 Schritt 5: Beispiel-Dialog

In [10]:

fragen = [
    "Welche wissenschaftlichen Ereignisse sind für 2025 geplant?",
    "Gibt es Missionen der NASA in diesem Jahr?",
    "Wie hängen diese Ereignisse mit der Raumfahrt zusammen?"
]

for frage in fragen:
    antwort = rag_chain.run(frage)
    print(f"\n🙋 Frage: {frage}\n🤖 Antwort: {antwort}")


C:\Users\volodymyr\AppData\Local\Temp\ipykernel_26284\2755556264.py:8: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  antwort = rag_chain.run(frage)



🙋 Frage: Welche wissenschaftlichen Ereignisse sind für 2025 geplant?
🤖 Antwort: Die Vereinten Nationen haben 2025 zum Internationalen Jahr der Quantenwissenschaft und -technologie erklärt.

🙋 Frage: Gibt es Missionen der NASA in diesem Jahr?
🤖 Antwort: Ja, die NASA hat mehrere Missionen geplant, die 2025 starten sollen. Dazu gehören die Pandora-Mission, die im September 20 Sterne und ihre 39 Exoplaneten beobachten soll, und die IMAP-Sonde, die zum Lagrange-Punkt 1 starten soll, um interstellaren Staub zu sammeln und das Weltraumwetter zu untersuchen.

🙋 Frage: Wie hängen diese Ereignisse mit der Raumfahrt zusammen?
🤖 Antwort: Die NASA wird im September 2025 ihre Pandora-Mission starten, die darauf abzielt, 20 Sterne und ihre 39 Exoplaneten zu beobachten. Die IMAP-Sonde der NASA wird 2025 gestartet, um interstellaren Staub einzufangen.
